In [ ]:
# !pip install plotly
# !pip install lazypredict
#!pip install pandas-profiling

In [ ]:
import glob
import pandas as pd

csv_files = glob.glob("*(2).csv")
#csv_files = glob.glob("*w.csv")
print(csv_files)

dataset = pd.concat([pd.read_csv(file) for file in csv_files], ignore_index=True)
dataset.to_csv("dataset_normal.csv", index=False)
#dataset.to_csv("dataset_euro_raw.csv", index=False)


['200_wled_raw (2).csv', '10_wled_raw (2).csv', '500_wled_raw (2).csv', '20_wled_raw (2).csv', '50_wled_raw (2).csv', '100_wled_raw (2).csv']


In [ ]:
import pandas as pd
from lazypredict.Supervised import LazyClassifier
import plotly.express as px
#from pandas_profiling import ProfileReport

In [ ]:
df = pd.read_csv("/content/dataset_normal.csv")
# df_raw = pd.read_csv("/content/dataset_tcs_inr_raw.csv")
# uv = pd.read_csv("/content/dataset_tcs_inr_uv.csv")
# uv_raw = pd.read_csv("/content/dataset_tcs_inr_uv_raw.csv")

In [ ]:
df.head()


,R1,G1,B1,Clear1,Temperature1,Intensity1,R2,G2,B2,Clear2,Temperature2,Intensity2,class
0,189,147,102,475,3427.03,1032.2,225,299,272,776,None,None,200
1,189,147,101,474,3427.03,1032.2,225,298,272,776,None,None,200
2,189,147,102,474,3427.03,1032.2,224,296,269,770,None,None,200
3,189,147,102,474,3447.19,1032.2,225,298,272,775,None,None,200
4,189,147,101,474,3427.03,1032.2,225,297,271,774,None,None,200


In [ ]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3105 entries, 0 to 3104
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   R1            3105 non-null   int64 
 1   G1            3105 non-null   int64 
 2   B1            3105 non-null   int64 
 3   Clear1        3105 non-null   int64 
 4   Temperature1  3105 non-null   object
 5   Intensity1    3105 non-null   object
 6   R2            3105 non-null   int64 
 7   G2            3105 non-null   int64 
 8   B2            3105 non-null   int64 
 9   Clear2        3105 non-null   int64 
 10  Temperature2  3105 non-null   object
 11  Intensity2    3105 non-null   object
 12  class         3105 non-null   int64 
dtypes: int64(9), object(4)
memory usage: 315.5+ KB


In [ ]:
features = df.drop(['class'], axis=1)
labels = df['class']
classes = labels.unique()
columns = features.columns.tolist()
print(classes)
print(columns)

[200  10 500  20  50 100]
['R1', 'G1', 'B1', 'Clear1', 'Temperature1', 'Intensity1', 'R2', 'G2', 'B2', 'Clear2', 'Temperature2', 'Intensity2']


In [ ]:
df["class"].unique()

array([200,  10, 500,  20,  50, 100])

In [ ]:
'''
#Find outliers
print('Outliers')
for cls in classes:
  df_cls = df[df['class'] == cls]
  des_cls = df_cls.describe()
  print(f'\nFor {cls}')
  print(des_cls, '\n')
  for column in columns:
    q3 = des_cls[column][-2]
    q1 = des_cls[column][-4]
    iqr = q3 - q1
    up = q3 + (2.5 * iqr)
    lw = q3 - (2.5 * iqr)
    outliers = df_cls[(df_cls[column] < lw) | (df_cls[column] > up)]
    print(f'{column}: {len(outliers)}')
    print(f'up: {up}, lw: {lw}')
'''

"\n#Find outliers\nprint('Outliers')\nfor cls in classes:\n  df_cls = df[df['class'] == cls]\n  des_cls = df_cls.describe()\n  print(f'\nFor {cls}')\n  print(des_cls, '\n')\n  for column in columns:\n    q3 = des_cls[column][-2]\n    q1 = des_cls[column][-4]\n    iqr = q3 - q1\n    up = q3 + (2.5 * iqr)\n    lw = q3 - (2.5 * iqr)\n    outliers = df_cls[(df_cls[column] < lw) | (df_cls[column] > up)]\n    print(f'{column}: {len(outliers)}')\n    print(f'up: {up}, lw: {lw}')\n"

In [ ]:
df["class"].unique()

array([200,  10, 500,  20,  50, 100])

In [ ]:
# Remove outliers
# df.drop(df[(df['class'] == '100') & ((df['R1'] < 24.5) | (df['G1'] < 28.0) | (df['B1'] < 22.5))].index, inplace=True)
# df.drop(df[(df['class'] == '20') & ((df['R1'] < 41.0) | (df['G1'] < 33.0) | (df['B1'] < 16.5))].index, inplace=True)
# df.drop(df[(df['class'] == '200') & ((df['R1'] < 42.0) | (df['G1'] < 32.5) | (df['B1'] < 15.5))].index, inplace=True)
# df.drop(df[(df['class'] == '2000') & ((df['R1'] < 48.5) | (df['G1'] < 40.625) | (df['B1'] < 32.5) | (df['G1'] > 49.375))].index, inplace=True)
# df.drop(df[(df['class'] == '50') & ((df['R1'] < 26.5) | (df['G1'] < 38.5) | (df['B1'] < 25.5))].index, inplace=True)
# df.drop(df[(df['class'] == '10') & ((df['R1'] < 35) | (df['G1'] < 0) | (df['B1'] < 0))].index, inplace=True)

# df.drop(df[(df['class'] == '100') & (df['G1'] > 18.5)].index, inplace=True)
# df.drop(df[(df['class'] == '200') & ((df['R1'] < 25) | (df['G1'] > 19.5))].index, inplace=True)
# df.drop(df[(df['class'] == '500') & ((df['G1'] > 23.5))].index, inplace=True)

In [ ]:
df["class"].unique()

array([200,  10, 500,  20,  50, 100])

In [ ]:
# df[(df['class'] == '20') & (df['B1'] > 4)].index

In [ ]:
# df.to_csv('dataset_normal.csv', index=False)

In [ ]:
fig = px.scatter_3d(df, x='R1', y='G1', z='B1',
              color='class',animation_group= "class")
fig.update_traces(marker_size=4)
fig.show()

In [ ]:
fig = px.scatter_3d(df, x='R2', y='G2', z='B2',
              color='class',animation_group= "class")
fig.update_traces(marker_size=4)
fig.show()

In [ ]:
df["class"].unique()

array([200,  10, 500,  20,  50, 100])

In [ ]:
df = df[(df['class'] == '500') | (df['class'] == '2k') | (df['class'] == '200') | (df['class'] == '10') | (df['class'] == '20') | (df['class'] == '100') | (df['class'] == '50') | (df['class'] == 'blank')]

In [ ]:
df['class'].value_counts()

Series([], Name: class, dtype: int64)

In [ ]:
from sklearn.decomposition import PCA, IncrementalPCA
from sklearn import model_selection, tree, metrics, ensemble
import numpy as np
import matplotlib.pyplot as plt

In [ ]:

cols = ['R1', 'G1', 'B1','R2','G2','B2']
features = df.drop(columns=['class'])[cols]
label = df['class']
X, x, Y, y = model_selection.train_test_split(features, label, stratify=label, test_size=0.1, random_state=42)

ValueError: ignored

In [ ]:
features


In [ ]:
label.value_counts()

In [ ]:
clf = LazyClassifier()

predictions, scores = clf.fit(X, x, Y, y)

In [ ]:
predictions

# <h1>**training**

In [ ]:

cols = ['R1', 'G1', 'B1','R2','G2','B2']
features = df.drop(columns=['class'])[cols]
label = df['class']
X, x, Y, y = model_selection.train_test_split(features, label, stratify=label, test_size=0.2, random_state=42)

In [ ]:
classes = df["class"].unique().tolist()
classes.sort()
print(classes)   # Create classes list in pico program

In [ ]:
features

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Create a decision tree classifier
classifier = DecisionTreeClassifier()

# Train the classifier on the training data
classifier.fit(X, Y)

# Make predictions on the test data
y_pred = classifier.predict(x)


In [ ]:
x.describe()

In [ ]:
# Evaluate the classifier
accuracy = accuracy_score(y, y_pred)
print("Accuracy:", accuracy)

In [ ]:

print(
    f"Classification report for classifier {classifier}:\n"
    f"{metrics.classification_report(y, y_pred)}\n"
)

In [ ]:
!pip install m2cgen
!pip install python-minimizer

In [ ]:
!pip install m2cgen


In [ ]:
import sys
import m2cgen as m2c

sys.setrecursionlimit(999999999)
# Convert model to python code
code = m2c.export_to_python(classifier)
with open("classifier_normal_rgb.py", "w") as fp:
  fp.write(code)

In [ ]:
code

In [ ]:
!python-minimizer classifier_normal_rgb.py -o classifier_norm_rgb.py

In [ ]:
from classifier_norm_rgb import classify

In [ ]:

pred = classify([6,16,18,9,22,29])

print(pred)

print(len(pred))

print(classes)

CREATING A SVC


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import io
import pandas as pd
from google.colab import files

In [ ]:
X = df.drop('class', axis=1)
y = df['class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

svc_classifier = LinearSVC()

svc_classifier.fit(X_train, y_train)

In [ ]:
y_pred = svc_classifier.predict(X_test)
print("\nAccuracy:", accuracy_score(y_test, y_pred)*100)

In [ ]:
import m2cgen as m2c
code = m2c.export_to_python(svc_classifier, function_name="classify")
code

with open("LINEARsvc.py","w") as fp:
  fp.write(code)





In [ ]:
# predict function of the model
import pandas as pd
import numpy as np

# test the classifier using predict, but w an i/p query
input = np.array([29,	42,	31,	51,	125,	93]).reshape(1,-1)
# 29	42	31	51	125	93
pred = svc_classifier.predict(input)
print("prediction result: {}".format(pred))
pred

In [ ]:

from LINEARsvc import classify
input = [37,60,44,51,122,93]
pred = classify(input)


print(pred)
print(classes)


In [ ]:
'''
input = np.array([7, 17, 16, 36, 52, 31])
input
'''

In [ ]:
input = np.array([14,23,17,130,255,255]).reshape(1, -1)
# pred = classify(input[0])
pred=classify([29	,42,	31,	51,	125,	93])
print(pred)
print("prediction result: {}".format(pred))

In [ ]:
res=pred.index(max(pred))
print("index of class mapped",res)
mapped_class = classes[res]
print("Mapped class:", mapped_class)

In [ ]:
classes

In [ ]:
input

In [ ]:
input[0]

In [ ]:
ip_data=[14,23,17,130,255,255]
input_data_reshaped=([[ip_data]])
pred=classify(input[0])
pred

In [ ]:
print(classes)